# Introduction to Finite Frames and RIC-POVM's

Maturing out of our reflexive habit of working with orthonormal bases, we can instead work with an overcomplete basis or "frame," which can have as many elements as we like. In fact, with probability 1, any randomly chosen set of vectors ($n \ge d$) will form a frame.

Let's just think about $d=2$ real vector space. Why not a frame made of 5 vectors?

In [11]:
import numpy as np
import scipy as sc
import scipy.linalg
np.set_printoptions(precision=4, suppress=True)

d, n = 2, 5
R = np.random.randn(d, n); R

array([[-1.0498,  0.4192,  0.3598, -0.3203, -1.1067],
       [ 0.5574, -0.4075,  2.0128,  1.245 ,  1.0094]])

We can form the Gram matrix, aka the matrix of inner products:

In [12]:
G = R.T @ R; G

array([[ 1.4128, -0.6672,  0.7443,  1.0303,  1.7244],
       [-0.6672,  0.3418, -0.6693, -0.6416, -0.8752],
       [ 0.7443, -0.6693,  4.1809,  2.3907,  1.6335],
       [ 1.0303, -0.6416,  2.3907,  1.6527,  1.6112],
       [ 1.7244, -0.8752,  1.6335,  1.6112,  2.2436]])

One condition for everything being kosher is that the Gram matrix ought to be rank-$d$.

In [13]:
np.round(np.linalg.svd(G)[1], decimals=4)

array([7.5904, 2.2413, 0.    , 0.    , 0.    ])

We should also check out:

In [14]:
R @ R.T

array([[ 2.7346, -1.5476],
       [-1.5476,  7.0971]])

We'll have a frame if the latter has non-0 determinant aka is full rank. This means that the rows of $R$ are linearly independent.

In [15]:
np.linalg.det(R @ R.T)

17.012389286631418

By the way, the technical definition of a frame is a set of vectors $\{v\}$, for which there are constants $a, b > 0$ such that for any vector $u$:

$$ a ||u||^2 \leq \sum_{i} |\langle u|v_{i}\rangle|^2 \leq b||u||^2 $$

The $a$ and $b$ are called frame bounds, and this definition is always satisfied if the aforementioned conditions are true. We won't be using this definition too much. 

<hr>

## Analysis and Synthesis

Suppose we have some 2D point:

In [16]:
point = np.random.randn(2); point

array([-0.3741, -1.3575])

We can expand it in terms of our frame vectors:

In [17]:
c = R.T @ point; c

array([-0.364 ,  0.3963, -2.867 , -1.5703, -0.9563])

 `R.T` is often called the "analysis" operator.
 
How do we recover the original point, though? 

We use the dual frame, obtained by taking the pseudo-inverse of `R`. 

The inner products with the frame vectors turn out to be the weights on the dual basis vectors, which via summation recover the original point.

In [18]:
D = np.linalg.pinv(R); D

array([[-0.3872, -0.0059],
       [ 0.1378, -0.0274],
       [ 0.3332,  0.3563],
       [-0.0204,  0.171 ],
       [-0.3698,  0.0616]])

In [19]:
sum([c[i]*D[i] for i in range(n)])

array([-0.3741, -1.3575])

Or:

In [20]:
D.T @ c

array([-0.3741, -1.3575])

`D.T` is often called the "synthesis" operator.

It works because:

In [21]:
D.T @ R.T 

array([[1., 0.],
       [0., 1.]])

Conversely, we could have expanded in the dual basis:

In [22]:
c = D @ point; c

array([ 0.1529, -0.0144, -0.6083, -0.2245,  0.0548])

In [23]:
R @ c

array([-0.3741, -1.3575])

<hr>

Another way of thinking about this "recovery" procedure: instead, we could take the pseudo-inverse of the Gram matrix. 

In [24]:
Ginv = np.linalg.pinv(G); Ginv

array([[ 0.15  , -0.0532, -0.1311,  0.0069,  0.1429],
       [-0.0532,  0.0197,  0.0362, -0.0075, -0.0527],
       [-0.1311,  0.0362,  0.238 ,  0.0541, -0.1013],
       [ 0.0069, -0.0075,  0.0541,  0.0297,  0.0181],
       [ 0.1429, -0.0527, -0.1013,  0.0181,  0.1406]])

If we have our frame coordinates:

In [25]:
c = R.T @ point; c

array([-0.364 ,  0.3963, -2.867 , -1.5703, -0.9563])

Acting with $G^{-1}$ will give us the coordinates in the dual basis:

In [26]:
Ginv @ c

array([ 0.1529, -0.0144, -0.6083, -0.2245,  0.0548])

So that:

In [27]:
R @ Ginv @ c

array([-0.3741, -1.3575])

In other words, using $G^{-1}$, we can avoid explicitly invoking the dual basis.

<hr>

## Tight Frames

We might consider restrictions on our frames. Different kinds of frames are useful for different purposes. 

For example, everyone loves tight frames. We can get one via a generalization of a polar decomposition for rectangular matrices. Starting with a random $d \times n$ matrix $R$, we decompose as $R = UH$, where the rows of $U$ are orthonormal, and $H$ is Hermitian. Then we throw out the $H$ part.

In [28]:
R = np.random.randn(d, n)
R = sc.linalg.polar(R)[0]; R

array([[ 0.7192,  0.6138,  0.1819, -0.2684,  0.0295],
       [-0.3509, -0.0555,  0.4236, -0.8042, -0.2183]])

In [29]:
R.T @ R

array([[ 0.6404,  0.4609, -0.0178,  0.0892,  0.0978],
       [ 0.4609,  0.3798,  0.0882, -0.1201,  0.0302],
       [-0.0178,  0.0882,  0.2125, -0.3895, -0.0871],
       [ 0.0892, -0.1201, -0.3895,  0.7187,  0.1676],
       [ 0.0978,  0.0302, -0.0871,  0.1676,  0.0485]])

The point is that:

In [30]:
R @ R.T

array([[ 1., -0.],
       [-0.,  1.]])

So the frame is the same as its dual! In other words, the analysis operator is `R.T` and the synthesis operator is `R`.

In [31]:
point

array([-0.3741, -1.3575])

In [32]:
c = R.T @ point; c

array([ 0.2073, -0.1543, -0.6431,  1.192 ,  0.2853])

In [33]:
R @ c

array([-0.3741, -1.3575])

More generally, any frame for which $R R^{\dagger}$ is proportional to the identity is a "tight frame," so that the dual frame is proportional to the original frame. In terms of the "frame bounds" from before, a tight frame means that $a=b$. And when $a=b=1$, this is known as a "Parseval frame."

Also, for a tight frame, the Gram matrix is a rank-$d$ projector.

In [106]:
np.linalg.eig(R.T @ R)[0]

array([1., 0., 1.])

<hr>

## Unit Norm Tight Frames

We might also be interested not just in tight frames, but in "unit norm tight frames". As the name suggests, each vector has unit norm. 

One easy way to get one is to alternate between "tightening" and "normalizing" our vectors until we're satisfied.

In [34]:
def random_funtf(d, n, rtol=1e-15, atol=1e-15):
    R = np.random.randn(d, n)
    done = False
    while not (np.allclose(R @ R.T, (n/d)*np.eye(d), rtol=rtol, atol=atol) and\
               np.allclose(np.linalg.norm(R, axis=0), np.ones(n), rtol=rtol, atol=atol)):
        R = sc.linalg.polar(R)[0]
        R = np.array([state/np.linalg.norm(state) for state in R.T]).T
    return R

In [35]:
R = random_funtf(2, 5); R

array([[-0.9986, -0.8349,  0.0026, -0.8914, -0.1059],
       [-0.0524, -0.5505, -1.    ,  0.4533,  0.9944]])

In [36]:
R.T @ R

array([[ 1.    ,  0.8626,  0.0498,  0.8664,  0.0536],
       [ 0.8626,  1.    ,  0.5483,  0.4947, -0.459 ],
       [ 0.0498,  0.5483,  1.    , -0.4556, -0.9947],
       [ 0.8664,  0.4947, -0.4556,  1.    ,  0.5451],
       [ 0.0536, -0.459 , -0.9947,  0.5451,  1.    ]])

In [37]:
R @ R.T

array([[2.5, 0. ],
       [0. , 2.5]])

Notice that we've had to demand $RR^{\dagger}$ to be $\frac{n}{d} I$.

In [38]:
point

array([-0.3741, -1.3575])

In [39]:
c = R.T @ point; c

array([ 0.4447,  1.0596,  1.3565, -0.2819, -1.3103])

In [40]:
(d/n)*R @ c

array([-0.3741, -1.3575])

<hr>

## Equiangular Frames

As I say, there are many kinds of frames that people are interested in, which have various geometric and/or information theoretic properties.

One of the classics of the genre are the so-called "equiangular frames," which are made up of vectors which form equal angles with each other (or more properly, the absolute values of the inner products are all the same).

These are great for many reasons, not least because they provide very resilient representations of data even if you should lose a few components.

There are, however, geometrical restrictions on the maximum number of equiangular lines you can have in a given dimension. For example, in $d=2$, you can have a maximum of $3$ equiangular lines. 

The $3$ are easy to construct:

In [41]:
R = np.array([[r.real, r.imag] for r in [np.exp(2j*np.pi*i/3) for i in range(3)]]).T; R

array([[ 1.   , -0.5  , -0.5  ],
       [ 0.   ,  0.866, -0.866]])

In [42]:
R.T @ R

array([[ 1. , -0.5, -0.5],
       [-0.5,  1. , -0.5],
       [-0.5, -0.5,  1. ]])

In [43]:
R @ R.T

array([[1.5, 0. ],
       [0. , 1.5]])

The 3 lines form the diagonals of a hexagon. 

<img src="img/hexagon.jpg">

<hr>

In three dimensions, an equiangular frame is provided by the icosahedron, which has 20 faces and 12 vertices: its six diagonals provide the frame vectors.

<img width=300 src="img/icosahedron.png">

In [44]:
def circular_shifts(v): 
    """
    Returns all shifted versions of a vector, e.g.:
        [1,2,3]
        [3,1,2]
        [2,3,1]
    """
    shifts = [v]
    for i in range(len(v)-1):
        u = shifts[-1][:]
        u.insert(0, u.pop()) 
        shifts.append(u)
    return shifts

def icosahedron_frame():
    """
    Returns equiangular frame based on the icosahedron in three dimensions.
    """
    phi = (1+np.sqrt(5))/2 # golden ratio
    # 12 vertices provided by circular shifts of the four vectors
    vertices = [np.array(v) for v in 
                   circular_shifts([0, 1, phi]) + \
                   circular_shifts([0, -1, -phi]) + \
                   circular_shifts([0, 1, -phi]) + \
                   circular_shifts([0, -1, phi])]
    # throw out the antipodal vectors
    keep = []
    for i, a in enumerate(vertices):
        for j, b in enumerate(vertices):
            if i != j and np.allclose(a, -b) and j not in keep:
                keep.append(i)
    # normalize
    return np.array([e/np.linalg.norm(e) for i, e in enumerate(vertices) if i in keep]).T

In [45]:
R = icosahedron_frame(); R

array([[ 0.    ,  0.8507,  0.5257,  0.    , -0.8507,  0.5257],
       [ 0.5257,  0.    ,  0.8507,  0.5257,  0.    , -0.8507],
       [ 0.8507,  0.5257,  0.    , -0.8507,  0.5257,  0.    ]])

In [46]:
R.T @ R

array([[ 1.    ,  0.4472,  0.4472, -0.4472,  0.4472, -0.4472],
       [ 0.4472,  1.    ,  0.4472, -0.4472, -0.4472,  0.4472],
       [ 0.4472,  0.4472,  1.    ,  0.4472, -0.4472, -0.4472],
       [-0.4472, -0.4472,  0.4472,  1.    , -0.4472, -0.4472],
       [ 0.4472, -0.4472, -0.4472, -0.4472,  1.    , -0.4472],
       [-0.4472,  0.4472, -0.4472, -0.4472, -0.4472,  1.    ]])

In [47]:
R @ R.T

array([[2., 0., 0.],
       [0., 2., 0.],
       [0., 0., 2.]])

<hr>

Check out <a href="examples/frame_viz.py">frame_viz.py</a> to play around with different kinds of 2D frames! 

You'll be able to drag around a point in 2D. Press `c` to show standard Cartesian axes, and the projections of the point onto them. Press `f` to show the frame vectors and the projections onto them. Press `d` to show the dual vectors and the projections onto them. Press `q` to show the reconstruction of the point by a weighted sum of frame vectors (provided by the inner products with the dual vectors). Press `p` to show the reconstruction of the point by a weighted sum of dual vectors (provided by the inner products with the frame vectors).

Note that to get the projections right, we need to divide frame (dual) coordinates by the norm squared of the corresponding frame (dual) vectors. 

Because:

In [48]:
R = np.random.randn(2, 3); R

array([[ 0.116 ,  0.7711, -1.794 ],
       [ 1.193 ,  0.1818, -0.6445]])

In [49]:
point

array([-0.3741, -1.3575])

If we want to project the point onto the three frame vectors, we can use the projectors formed from the outer products $rr^{T}$, where $r$ is unit norm. This amounts to dividing by $||r||^2$.

In [50]:
np.array([np.outer(r/np.linalg.norm(r), r/np.linalg.norm(r))@point for r in R.T])

array([[-0.1343, -1.3808],
       [-0.6576, -0.155 ],
       [-0.7633, -0.2742]])

Meanwhile, we have our inner products:

In [51]:
c = R.T @ point; c

array([-1.6628, -0.5352,  1.546 ])

To recover the projections, we multiply each of the inner products by the vector, divided by the norm squared.

In [52]:
np.array([c[i]*R.T[i]/np.linalg.norm(R.T[i])**2 for i in range(3)])

array([[-0.1343, -1.3808],
       [-0.6576, -0.155 ],
       [-0.7633, -0.2742]])

Finally, in the case of 2D, we have another way of visualizing things. Press `3` to check it out.

We can interpret the three inner products with the frame vectors as a point in 3D. 

In [53]:
c

array([-1.6628, -0.5352,  1.546 ])

Moreover, `D` will be a $3 \times 2$ matrix: we can interpret its 2 columns as specifying two vectors in 3D.

In [54]:
D = np.linalg.pinv(R); D

array([[-0.2929,  0.8619],
       [ 0.2317, -0.0805],
       [-0.4768,  0.0212]])

We can project our 3D point onto these two vectors:

In [55]:
[np.outer(D.T[i], D.T[i])/np.linalg.norm(D.T[i])**2 @ c for i in range(2)]

[array([ 0.2987, -0.2363,  0.4862]), array([-1.5604,  0.1457, -0.0383])]

But this is just the same as weighting each of the two vectors by the Cartesian coordinates of the original point!

In [56]:
[point[i]*D.T[i]/np.linalg.norm(D.T[i])**2 for i in range(2)]

[array([ 0.2987, -0.2363,  0.4862]), array([-1.5604,  0.1457, -0.0383])]

In other words, the three inner products with the frame vector specify a point in 3D. The dual basis specifies two linearly independent axes in 3D. The projection of the 3D point onto these two axes gives two coordinates: these are proportional to the original 2D point in Cartesian coordinates.

<hr>

## RIC-POVM's


Now consider that $3$ is also the dimension of the space of $2 \times 2$ symmetric matrices. 

Indeed, we might be interested in this fact if we wanted to do real vector space quantum mechanics. 

In real quantum mechanics, pure states are given by 2D vectors $\mid \psi \rangle = \begin{pmatrix}x \\ y \end{pmatrix}$ such that $x^2 + y^2 = 1$, aka points on the circle; or better yet: rank-1 density matrices: $\mid \psi \rangle \langle \psi \mid = \begin{pmatrix}x^2 & xy \\ xy & y^2 \end{pmatrix}$. 

More generally, a density matrix might be a higher rank statistical mixture. Any symmetric, positive semi-definite matrix with unit trace will do.

Now here's the big reveal: it turns out that tight frames are just rank-1 POVM's in disguise. 

Recall that a POVM, or positive operator valued measure, is a set of positive semi-definite matrices that sum to the identity: $\sum_{i} E_{i} = I$. Each "element" represents an outcome to a generalized quantum measurement that concretely involves entangling your quantum system to an apparatus (with a dimensionality equal to the number of POVM elements) and doing a standard (Von Neumann) measurement on the latter. In the case of a rank-1 POVM, the outcome states of the original sytem will correspond to the normalized POVM elements.

A POVM can have any number of elements (well, I suppose at least two!): but we're interested in "informationally complete" POVM's, which have as many elements as the dimensions of the relevant operator space, aka the space of density matrices, which for real vector space is $\frac{d(d+1)}{2}$. (In complex quantum mechanics, we use not symmetric matrices, but Hermitian matrices, which have $d^2$ parameters.) 

If we have enough elements to span the operator space, then the vector of probabilities for each outcome suffice to completely determine the quantum state. And so, among other things, we can do quantum tomography, treating pure states and mixed states on an equal footing, by repeating one measurement over and over again on many copies of a system, instead of having to do different, complementary measurements.

We can upgrade a tight frame into a rank-1 POVM by taking the outer products of the vectors with themselves. (And by the way, it's not at all necessary that POVM elements be rank-1: those we'll be considering here, however, will be.) The POVM condition, as you can see, is satisfied since $RR^{\dagger}$ amounts to $\sum_{i} |r\rangle\langle r|$.

In [57]:
d, n = 2, 3
R = sc.linalg.polar(np.random.randn(d, n))[0]
R @ R.T

array([[ 1., -0.],
       [-0.,  1.]])

In [58]:
E = [np.outer(r, r) for r in R.T]
sum(E)

array([[ 1., -0.],
       [-0.,  1.]])

Given a random density matrix:

In [59]:
def rand_dm(d, n=2):
    # n controls the degree of mixedness
    A = np.random.randn(d, n)
    rho = A @ A.T
    return rho/rho.trace()

rho = rand_dm(2); rho

array([[ 0.9548, -0.1295],
       [-0.1295,  0.0452]])

We can expand it in terms of POVM elements:

In [60]:
p = np.array([(e @ rho).trace() for e in E]); p

array([0.7141, 0.2213, 0.0645])

In [61]:
sum(p)

1.0

The components sum to 1: they are the probabilities for the outcomes to this experiment. 

<hr>

Now how do we get the density matrix back? The dual basis, of course! But how do we find it?

Given a matrix $A$, denote by $|A\rangle \rangle$ the vector you get by reshaping the matrix into a vector. 

(If you want to be fancy, you can think of this like $\sum_{i} (A \otimes I)\mid i \rangle\mid i \rangle$, or as $A$ acting on the left on the maximally entangled "cup" state on the doubled Hilbert space, $H_{d} \otimes H_{d}$).

Then form the "frame operator":

$F = \sum_{i} \mid E_{i} \rangle \rangle \langle \langle E_{i} \mid = \sum_{i} E_{i} \otimes E_{i}$

Next, take its (pseudo) inverse. Finally, $F^{-1}\mid E_{i}\rangle \rangle = \mid \tilde{E}_{i} \rangle \rangle$, where $\tilde{E}$ is the dual basis. 

In [62]:
F = sum([np.kron(e, e) for e in E])
Finv = np.linalg.pinv(F)
D = [(Finv @ e.reshape(d**2)).reshape(d,d) for e in E]

Then, we can recover the original density matrix by weighting the elements of the dual basis by the probabilities and summing:

In [63]:
rho

array([[ 0.9548, -0.1295],
       [-0.1295,  0.0452]])

In [64]:
sum([p[i]*D[i] for i in range(n)])

array([[ 0.9548, -0.1295],
       [-0.1295,  0.0452]])

Conversely, we could expand in dual elements:

In [65]:
q = np.array([(e @ rho).trace() for e in D]); q

array([ 1.4418, -0.3911,  0.0653])

And recover the original density matrix by weighting POVM elements and summing

In [66]:
sum([q[i]*E[i] for i in range(n)])

array([[ 0.9548, -0.1295],
       [-0.1295,  0.0452]])

By the way, the dual basis of an informationally complete POVM <a href="https://physics.stackexchange.com/questions/588467/dual-of-ic-povm-cannot-be-positive">doesn't form a POVM itself</a>. Indeed, its elements must be indefinite!

<hr>

Alternatively, instead of working with the dual basis directly, we could play a similar trick from before. 

Instead of taking $G^{-1}$, now we take $(G \circ G )^{-1}$ where $\circ$ is the Hadamard (element-wise) square. Instead of expanding in the dual basis, we can act with $(G \circ G )^{-1}$ on the vector of probabilities to get the proper weights for POVM elements required to recover $\rho$.

In [67]:
rho

array([[ 0.9548, -0.1295],
       [-0.1295,  0.0452]])

In [68]:
p

array([0.7141, 0.2213, 0.0645])

In [69]:
G = R.T @ R
Gsq_inv = np.linalg.inv(G**2)

In [70]:
q = Gsq_inv @ p; q

array([ 1.4418, -0.3911,  0.0653])

We can now get our density matrix back:

In [71]:
sum([q[i]*E[i] for i in range(n)])

array([[ 0.9548, -0.1295],
       [-0.1295,  0.0452]])

<hr>

But we can do a little bit more, which will prove useful.

We could form a matrix of conditional probabilities supposing we repeat our POVM twice. In other words, a matrix of conditional probabilities: 

$$ Pr(E_{i}|E_{j}) = tr(E_{i}\frac{E_{j}}{tr(E_{j})})$$

After the first measurement, we end up in some $\frac{E_{j}}{tr(E_{j})}$, and then we look at the probabilities that we get some $E_{i}$.


In [72]:
E_given_E = np.array([[(E[i]@(E[j]/E[j].trace())).trace() for j in range(n)] for i in range(n)]); E_given_E

array([[0.7371, 0.5634, 0.0154],
       [0.2432, 0.3182, 0.0399],
       [0.0197, 0.1184, 0.9448]])

Notice that this related to $(G \circ G )$, except for the fact that we divide by $tr(E_{j})$. 

We can take its inverse to obtain the "Born matrix" $\Phi$:

In [73]:
phi = np.linalg.inv(E_given_E); phi

array([[ 3.3097, -5.9342,  0.1966],
       [-2.5616,  7.786 , -0.2869],
       [ 0.2519, -0.8518,  1.0903]])

What's the point? If we take our density matrix:

In [74]:
rho

array([[ 0.9548, -0.1295],
       [-0.1295,  0.0452]])

And its probabilities:

In [75]:
p

array([0.7141, 0.2213, 0.0645])

Then we can form:

In [76]:
q = phi @ p; q

array([ 1.0627, -0.1244,  0.0617])

In [77]:
sum(q)

1.0

This is nice, because by acting on our vector of probabilities with $\Phi$, we get back a vector of *quasi-probabilities*: while the components might be negative, they nevertheless sum to 1. And this will prove to be useful.

We can then get our density matrix back by summing over normalized elements of our POVM, weighted by the quasi-probabilities.

In [78]:
sum([q[i]*E[i]/E[i].trace() for i in range(n)])

array([[ 0.9548, -0.1295],
       [-0.1295,  0.0452]])

Moreover, we can also obtain this matrix $\Phi$ from the dual elements. Actually, it's nicer to renormalize the dual elements first so that they have the same weights as the POVM elements.

In [79]:
D = [E[i].trace()*D[i] for i in range(n)]
sum(D)

array([[ 1., -0.],
       [-0.,  1.]])

In [80]:
D_given_D = np.array([[(D[i]@(D[j]/D[j].trace())).trace() for j in range(n)] for i in range(n)]); D_given_D

array([[ 3.3097, -5.9342,  0.1966],
       [-2.5616,  7.786 , -0.2869],
       [ 0.2519, -0.8518,  1.0903]])

Expanding in renormalized dual elements gives us our quasiprobabilities:

In [81]:
q = np.array([(d @ rho).trace() for d in D]); q

array([ 1.0627, -0.1244,  0.0617])

And as before:

In [82]:
sum([q[i]*E[i]/E[i].trace() for i in range(n)])

array([[ 0.9548, -0.1295],
       [-0.1295,  0.0452]])

And symmetrically:

In [83]:
sum([p[i]*D[i]/D[i].trace() for i in range(n)])

array([[ 0.9548, -0.1295],
       [-0.1295,  0.0452]])

<hr>

## Deforming the Law of Total Probability

Okay, but why should we care?

We're going to consider two measurements on our quantum system, $A$ and $B$, where $A$ is an IC-POVM, and $B$ could be a standard Von Neumann measurement or a POVM, doesn't matter. Let's say it's an IC-POVM.

In [84]:
d, n = 2, 3
A = [np.outer(r, r) for r in sc.linalg.polar(np.random.randn(d, n))[0].T]
B = [np.outer(r, r) for r in sc.linalg.polar(np.random.randn(d, n))[0].T]

In [85]:
rho

array([[ 0.9548, -0.1295],
       [-0.1295,  0.0452]])

We can of course nail down our quantum state by its probabilities on measurement $A$.

In [86]:
p_A = np.array([(a @ rho).trace() for a in A]); p_A

array([0.0382, 0.8644, 0.0974])

Now we're going to compare two sequences of events.

First: we measure $A$, and then measure $B$. Before we look at the outcome for $A$, however, what probabilities should we assign to the outcomes of $B$?

To figure this out, we could use a matrix of conditional probabilities:

In [87]:
B_given_A = np.array([[(b@a/a.trace()).trace() for a in A] for b in B]); B_given_A

array([[0.0964, 0.0075, 0.1241],
       [0.888 , 0.1795, 0.4162],
       [0.0157, 0.813 , 0.4597]])

These are the conditional probabilities for each of $B$'s outcomes, given each outcome state of $A$. This gives us conditional probabilities $Pr(B_{i}|A_{j})$. 

We end up with a column stochastic matrix:

In [88]:
np.sum(B_given_A, axis=0)

array([1., 1., 1.])

We can then use the law of total probability: $Pr(B_{i}) = \sum_{j} Pr(B_{i}|A_{j}) Pr(A_{j})$.

In [89]:
p_AB = B_given_A @ p_A; p_AB

array([0.0222, 0.2296, 0.7482])

In [90]:
sum(p_AB)

0.9999999999999997

We can check this. After the measurement of $A$, we should regard our system's state as a statistical mixture of $A$'s outcomes, weighted by the probability of their occurance:

In [91]:
rho_post_A = sum([p_A[i]*a/a.trace() for i, a in enumerate(A)]); rho_post_A

array([[ 0.8742, -0.0714],
       [-0.0714,  0.1258]])

We can then get the $B$ probabilities for this density matrix by taking inner products with $B$'s elements.

In [92]:
p_AB2 = np.array([(b @ rho_post_A).trace() for b in B]); p_AB2

array([0.0222, 0.2296, 0.7482])

In [93]:
sum(p_AB2)

0.9999999999999996

We get the same probabilities. 

<hr>

To recap: we did measurement of $A$, but didn't look at the answer. We just wanted the probabilities for the future measurement $B$. So we used the law of total probability: $p(B_{i}) = \sum_{j} p(B_{i}|A_{j}) p(A_{j})$.

Okay. 

The second experiment is that we leave $A$ hypothetical, but instead just do the $B$ measurement. We know what the answer should be: we just expand in terms of $B$.

In [94]:
p_B = np.array([(b @ rho).trace() for b in B]); p_B

array([0.0154, 0.1457, 0.8389])

But since our probabilities on $A$ completely nail down the state, there ought to be a way of getting these latter probabilities in terms of them...

It's easy! We simply form the "Born matrix" $\Phi$ from before:

In [95]:
phi = np.linalg.inv([[(a1 @ a0/a0.trace()).trace() for a0 in A] for a1 in A])

And instead of doing:

In [96]:
B_given_A @ p_A 

array([0.0222, 0.2296, 0.7482])

We do:

In [97]:
B_given_A @ phi @ p_A 

array([0.0154, 0.1457, 0.8389])

Hence the name the "Born matrix": it allows us to recover the Born rule. Notice how everything is still formulated in terms of a) the conditional probability matrix for B's outcomes given A's outcomes, and b) the probability vector for A's outcomes. 

One can now see the usefulness of:

In [98]:
q = phi @ p_A; q

array([-0.0942,  0.9549,  0.1393])

In [99]:
sum(q)

0.9999999999999999

The fact that $\Phi$ takes us to quasiprobabilities makes sure we play nicely with the conditional probability matrix.

Another way of thinking about this is to form a "conditional quasiprobability matrix":

$$ Qr(B_{i}|\tilde{A}_{j}) = tr(B_{i}\frac{\tilde{A}_{j}}{tr(\tilde{A}_{j})})$$


In [100]:
FA = sum([np.kron(e, e) for e in A])
FAinv = np.linalg.pinv(FA)
DA = [e.trace()*(FAinv @ e.reshape(d**2)).reshape(d,d) for e in A]
B_given_DA = np.array([[(B[i]@(DA[j]/DA[j].trace())).trace() for j in range(n)] for i in range(n)]); B_given_DA

array([[ 0.0436, -0.0101,  0.2298],
       [ 1.664 ,  0.1645, -0.6158],
       [-0.7076,  0.8456,  1.386 ]])

Then:

In [101]:
B_given_DA @ p_A

array([0.0154, 0.1457, 0.8389])

Indeed:

In [102]:
np.allclose(B_given_DA, B_given_A@phi)

True

Perhaps more concretely:

If we perform POVM $A$, then:

$$ \rho \rightarrow \rho^{\prime} = \sum_{i} Pr(A_{i}|\rho)\frac{A_{i}}{tr(A_{i})}$$

If we *don't* perform POVM $A$, then:

$$ \rho \rightarrow \sum_{i} Pr(A_{i}|\rho)\frac{\tilde{A}_{i}}{tr(\tilde{A}_{i})}$$

Which is just $\rho$! 

If we perform POVM $A$, and then POVM $B$:

$$ \rho \rightarrow \sum_{i} \Big{(} \sum_{j} Pr(B_{i}|A_{j}) Pr(A_{j}|\rho) \Big{)} \frac{B_{i}}{tr(B_{i})} = \sum_{i} Pr(B_{i}|\rho^{\prime}) \frac{B_{i}}{tr(B_{i})}$$

If we *don't* perform POVM $A$, and then do perform POVM $B$:

$$ \rho \rightarrow  \sum_{i} \big{(} \sum_{j} Qr(B_{i}|\tilde{A}_{j}) Pr(A_{j}|\rho) \big{)} \frac{B_{i}}{tr(B_{i})} =  \sum_{i} Pr(B_{i}|\rho)  \frac{B_{i}}{tr(B_{i})} $$

And if we did neither:

$$ \rho \rightarrow  \sum_{i} \big{(} \sum_{j} Qr(B_{i}|\tilde{A}_{j}) Pr(A_{j}|\rho) \big{)} \frac{\tilde{B}_{i}}{tr(\tilde{B}_{i})} =  \sum_{i} Pr(B_{i}|\rho)  \frac{\tilde{B}_{i}}{tr(\tilde{B}_{i})} $$

Which is just $\rho$!


<hr>

## Quantumness 

So in the first case, we did $A$, and afterwards we described our system as a mixed state of A's outcomes, weighted by the probability of them happening. No "coherence" between outcomes: there's just "classical ignorance." And we used the old school law of total probability to get the probabilities for the outcomes of $B$.

In the second case, we didn't do $A$: and so "quantum coherence" was preserved. But we could still get the probabilities for a $B$ measurement from the probabilities of the (hypothetical) $A$ measurement.

One way of thinking of this is that quantum mechanics encourages us to modify the law of total probability. Classically speaking, we could use the law of total probability whether or not we did an intermediate measurement of $A$ before $B$. But quantum mechanically, it matters. If we don't do $A$, we have to stick the Born matrix $\Phi$ in between the conditional probability matrix and the probability vector. In matrix notation, instead of $\vec{Pr(B_{i})} = \hat{Pr(B_{i}|A_{j})}\vec{Pr(A_{j})}$, we have $\vec{Pr(B_{i})} = \hat{Pr(B_{i}|A_{j})}\hat{\Phi}\vec{Pr(A_{j})}$.

Now, if $\Phi$ were the identity, then there would be no difference, as it were, between classical and quantum. No coherence, everything having "objective properties," law of total probability always valid, etc. The lesson of quantum mechanics is that: $\Phi$ is not the identity.

So you might wonder: what measurements are associated with a $\Phi$ such that $\Phi$ is *as close to the identity as possible*?

With this in mind, we form a quantity we'll call "quantumness": $Q = ||\Phi - I||$.

In [103]:
def quantumness(E):
    phi = np.linalg.inv([[(e1@e0/e0.trace()).trace() for e0 in E] for e1 in E])
    return np.linalg.norm(phi - np.eye(len(E)))

In [104]:
quantumness(A), quantumness(B)

(4.339944904790864, 17.066025672275895)

This quantity diagnoses the extent to which the law of total probability is deformed.


Now it's known that SIC-POVM's aka "symmetrically informationally complete POVM's," which correspond to equiangular lines, minimize quantumness. And in fact, the proof that they do so doesn't depend on the particulars of which norm you choose to measure the difference between $\Phi$ and the identity. 

In other words, and it makes sense, SIC-POVM's, whose elements all make the same angle with each other, have $\Phi$ matrices as close to the identity as possible. They are the quantum measurements that are "irreducibly quantum."

Now much of the study of SIC-POVM's takes place in the context of *real* quantum mechanics, I mean, complex vector space quantum mechanics. In this case, SIC-POVM's would correspond to *complex equiangular lines*, which are a bit more difficult to visualize. And to span the space of density matrices, you'd need $d^2$ of them for dimension $d$.

For example, in the case of $d=2$, the four elements of a SIC-POVM make up the vertices of a regular tetrahedron on the Bloch sphere. 

It's been conjectured, and widely believed, on the basis of a great deal of theoretical and computational effort, that SIC-POVM's exist in all complex dimensions. And conditional on this being true, then we know the irreducibly quantum measurements in any dimension: they are the SIC's.

<hr>

Here's the thing, though. 

As I've said: in real quantum mechanics, RIC-POVM's (real informationally complete POVM's) require $n=\frac{d(d+1)}{2}$ elements. As we've seen, this is fine for $d=2$: in this case, $n=3$, and we have the "Mercedes-Benz" frame. So too, all is well in $d=3$. Then $n=6$, and we have our icosahedron.

But then the shit hits the fan. 

Here's a table listing the known maximum number of equiangular lines in different real dimensions compared to the dimensionality of the symmetric operator space. 

I've starred the ones where SIC-POVM's can exist, namely: 2, 3, 7, 23 (so far as we know).

| d | max| n |
|---|-----|----------------|
| 2 |  3  |  3* |
| 3 |  6  |  6*|
| 4 |  6  |  10|
| 5 | 10  |  15|
| 6 | 16  |  21|
| 7 | 28 |  28*|
| 8 | 28 | 36|
| 9 | 28 | 45|
| 10 | 28 | 55|
| 11 | 28 | 66|
| 12 | 28 | 78|
| 13 | 28 | 91|
|14 | 28-30| 105|
|15 | 36 | 120|
|16 | 40-42| 136|
|17 | 48-50| 153|
|18 | 48-61| 171|
|19 | 72-76| 190|
|20 | 90-96| 210|
|21 | 126 | 231|
|22 | 176 | 253|
|23 | 276 | 276*|

(Incidentally, the famous Gerzon bound says there definitely can't be more that $\frac{d(d+1)}{2}$ equiangular lines in $\mathbb{R}^n$).

You might wonder: given that (unlike in the complex case), we definitely know there aren't SIC's in all real dimensions, what's the *next best thing*? Specifically, what are the RIC's with lowest quantumness in each real dimension where a SIC doesn't exist? 

It's an open question!

<hr>

## Bibliography

<a href="https://www.math.auckland.ac.nz/~waldron/Preprints/Frame-book/draft26-7.17.pdf">
An Introduction to Finite Tight Frames</a>

<a href="https://faculty.sites.iastate.edu/esweber/files/inline-files/book_final.pdf">Frames for Undergraduates</a>

<a href="http://www1.chapman.edu/~mbvajiac/conferences/chapman_lec03_slides.pdf">Slides on Frame Theory</a>
    
<a href="https://arxiv.org/abs/1311.3219">New bounds for equiangular lines</a>

<a href="https://www.lib.uidaho.edu/digital/etd/items/oldroyd_idaho_0089e_10817.html">Generalizations and Approximations of Equiangular Tight Frames</a>

<a href="https://arxiv.org/abs/1812.08762">The Varieties of Minimal Tomographically Complete Measurements</a>

<a href="https://arxiv.org/abs/1805.08721">Symmetric Informationally Complete Measurements Identify the Irreducible Difference between Classical and Quantum Systems</a>

<a href="https://arxiv.org/abs/quant-ph/0310075">Symmetric Informationally Complete Quantum Measurements</a>

<a href="https://arxiv.org/abs/1911.05809">Sporadic SICs and Exceptional Lie Algebras</a>